In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

import time
import os

import pandas as pd

In [15]:
company_list = pd.read_excel("iss_最終整理名單.xls")

In [16]:
company_list.head()

,transcript,id,Company Name,Exchange:Ticker,Watch Lists,Excel Company ID,CIK,Compustat GVKEY,LEI/CICI,Security Tickers,SIC Codes,SIC Codes (Primary Code Only),SIC Codes (Primary)
0,0,1,1st Source Corporation (NasdaqGS:SRCE),NasdaqGS:SRCE,executive_comp,IQ176404,0000034782,015199,-,NasdaqGS:SRCE,6022 State commercial banks; 6020 Commercial b...,6020,6020 Commercial banks
1,0,2,360 Communications Company,-,executive_comp,IQ348389,0001003959,062229,-,-,4812 Radiotelephone communications,4812,4812 Radiotelephone communications
2,1,3,3Com Corporation,-,executive_comp; iss_cusip,IQ24107,0000738076,010553,-,-,3577 Computer peripheral equipment; 3576 Compu...,3576,3576 Computer communications equipment
3,1,4,3D Systems Corporation (NYSE:DDD),NYSE:DDD,executive_comp; iss_cusip,IQ308402,0000910638; 0001023748,014898,5493003HHDOCCW4OP744,NYSE:DDD; WBAG:DDD; DB:SYV; BMV:DDD *; SWX:DDD...,3559 Special industry machinery; 3577 Computer...,3559,3559 Special industry machinery
4,1,5,"3M Cogent, Inc.",-,executive_comp,IQ4522840,0001289434,160498,-,-,7371 Computer programming services; 7373 Compu...,7373,7373 Computer integrated systems design


In [17]:
transcripts_Summary_href = "https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx"

In [18]:
def getBrowser(username=None, password=None):
    # start web browser
    browser=webdriver.Chrome()
    browser.get(transcripts_Summary_href)

    browser.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    browser.find_element_by_xpath('//*[@id="password"]').send_keys(password)
    browser.find_element_by_xpath('//*[@id="myLoginButton"]').click()
    
    return browser

In [19]:
def getHTML(browser, startID=0, endID=company_list.shape[0], IDs = []):

    first_half_XPath = '//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr['
    last_half_XPath = ']/td[3]/span/a'
    
    #for i in IDs:  # 自己選擇要爬哪些ID
    for i in range(startID, endID):

        if not company_list["transcript"][i]:
            continue

        # Get the IQID and ID of current company
        #
        # e.g. IQID of company "3Com Corporation" is IQ24107
        #      ID   of company "3Com Corporation" is 3
        #
        company_IQID = company_list["Excel Company ID"][i][2:]
        company_ID = company_list["id"][i]

        # Get the href of summary of all transcripts of current company
        #
        # e.g. https://www.capitaliq.com/CIQDotNet/Transcripts/Summary.aspx?companyId=24107
        #
        company_href = transcripts_Summary_href + "?companyId=" + str(company_IQID)

        # Go to that href
        browser.get(company_href)
        time.sleep(10)
        
        # Recheck if this company has no transcript
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))    
        if table_length == 1:
            continue

        # Get the total number of transcript 
        #
        # e.g. Extract "14" from "Viewing 1-14 of 14"
        #
        all_trans_numbers = None
        try:
            all_trans_numbers_XPath = first_half_XPath + str(table_length) + ']/td'
            all_trans_numbers = browser.find_element_by_xpath(all_trans_numbers_XPath).text
            all_trans_numbers = int(all_trans_numbers.split()[3])
        except:
            print(company_ID, "found no number")
            continue

        # Click 'View All' botton if it exist
        try:
            browser.find_element_by_link_text('View All').click()

            # 有時候 View All 還沒跑完就開始拉資料了，所以等一下，
            # 因為 transcripts 越多就要等越久，
            # 所以這裡我設需要等 (all_trans_numbers / 10) 秒
            time.sleep(all_trans_numbers / 10)
        except:
            pass


        # Get all of the hrefs and names of transcipts of company
        all_transcipts_href = []
        all_transcipts_name = []
        table_length = len(browser.find_elements_by_xpath('//*[@id="_transcriptsGrid__dataGrid"]/tbody/tr'))
        for j in range(2, table_length):

            full_XPath = first_half_XPath + str(j) + last_half_XPath

            try:
                page_href = browser.find_element_by_xpath(full_XPath).get_attribute('href')
                all_transcipts_href.append(page_href)

                name = browser.find_element_by_xpath(full_XPath).text
                all_transcipts_name.append(name)
            except:
                pass

        #print(all_trans_numbers, len(all_transcipts_href))

        # Check for missing transcipts 
        # 因為有些 transcipts 還沒被放上去只有文字標題，
        # 但是它還是被算在 all_trans_numbers 裡面，
        # 所以選擇 >10 確保沒有遺失 transcipts
        if all_trans_numbers - len(all_transcipts_href) > 10:
            print("not retrieve all trans in id :", company_ID)

        filepath = os.path.join('transcripts', str(company_ID))
        # Check if the folder exists. If not, create one
        if not os.path.exists(filepath):
            os.mkdir(filepath)

        for j in range(len(all_transcipts_href)):

            browser.get(all_transcipts_href[j])
            html = browser.page_source

            filename = os.path.join(filepath, all_transcipts_name[j].replace('/', '_')) + '.html'
            #print(filename)
            try:
                f = open(filename, "w")
                f.write(str(BeautifulSoup(html, 'html.parser').prettify()))
                f.close()
            except:
                pass

        if len(all_transcipts_href) != len(os.listdir(filepath)):
            print(len(all_transcipts_href) - len(os.listdir(filepath)), \
                    "transcript missed for id :", company_ID)

In [20]:
browser = getBrowser(username = "wanxin.wang13@imperial.ac.uk", password = "WWXwwx901021")

# Waiting for logging
time.sleep(10)

getHTML(browser)

TypeError: unsupported operand type(s) for -: 'list' and 'int'